# Clustering Crypto

In [1]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import altair as alt
from pathlib import Path

### Fetching Cryptocurrency Data

In [5]:
# Alternatively, use the provided csv file:
file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
crypto_df = pd.read_csv(file_path, index_col = 0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [7]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.loc[crypto_df["IsTrading"] == True]

In [9]:
# Remove the "IsTrading" column
crypto_df.drop("IsTrading", axis=1, inplace=True)

In [10]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace=True)


In [11]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.loc[crypto_df["TotalCoinsMined"] > 0]

In [12]:
# Drop rows where there are 'N/A' text values
crypto_df.isnull().sum()

CoinName           0
Algorithm          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

In [13]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_df = pd.DataFrame(data=crypto_df, columns=["CoinName"])

In [14]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop("CoinName", axis=1, inplace=True)

In [15]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Standardize data
crypto_scaled = StandardScaler().fit_transform(X)

### Reducing Dimensions Using PCA

In [17]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

crypto_pca = pca.fit_transform(crypto_scaled)

In [18]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)
pcs_df.head(10)

,PC 1,PC 2,PC 3
42,-0.323465,0.992356,-0.539580
404,-0.306844,0.992283,-0.540017
1337,2.304502,1.559265,-0.694671
BTC,-0.129944,-1.266724,0.098704
ETH,-0.153691,-2.027967,0.389313
LTC,-0.158225,-1.139292,0.043720
DASH,-0.399962,1.227708,-0.578743
XMR,-0.137547,-2.206674,0.435209
ETC,-0.152138,-2.028102,0.389301
ZEC,-0.165753,-1.807345,0.322140


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [23]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using altair
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
alt.Chart(df_elbow).mark_line().encode(x="k", y="inertia").interactive()


alt.Chart(...)

Running K-Means with `k=<your best value for k here>`

In [27]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)
# Predict clusters
predictions = model.predict(pcs_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
pcs_df["class"] = model.labels_
clustered_df = pd.concat([coin_df, crypto_df, pcs_df], axis=1)
clustered_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,class
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42,-0.323465,0.992356,-0.539580,0
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.306844,0.992283,-0.540017,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359,2.304502,1.559265,-0.694671,0
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000,-0.129944,-1.266724,0.098704,3
ETH,Ethereum,Ethash,PoW,1.076842e+08,0,-0.153691,-2.027967,0.389313,3
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000,-0.158225,-1.139292,0.043720,3
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000,-0.399962,1.227708,-0.578743,0
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0,-0.137547,-2.206674,0.435209,3
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000,-0.152138,-2.028102,0.389301,3
ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000,-0.165753,-1.807345,0.322140,3


### Visualizing Results

In [31]:
# Create a Scatter plot using alrair to visualize clusters
alt.Chart(clustered_df).mark_circle(size=60).encode(
    x="PC 1",
    y="PC 2",
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply"]
).interactive()

alt.Chart(...)

#### Table of Tradable Cryptocurrencies

In [42]:
# Table with tradable cryptos
display(clustered_df)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,class
42,42 Coin,Scrypt,PoW/PoS,0.000042,0.000042,-0.323465,0.992356,-0.539580,0
404,404Coin,Scrypt,PoW/PoS,1055.184902,532.000000,-0.306844,0.992283,-0.540017,0
1337,EliteCoin,X13,PoW/PoS,29279.424623,314159.265359,2.304502,1.559265,-0.694671,0
BTC,Bitcoin,SHA-256,PoW,17.927175,21.000000,-0.129944,-1.266724,0.098704,3
ETH,Ethereum,Ethash,PoW,107.684223,0.000000,-0.153691,-2.027967,0.389313,3
...,...,...,...,...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,1999.999995,2000.000000,2.511277,0.972521,-0.158933,0
GAP,Gapcoin,Scrypt,PoW/PoS,14.931046,250.000000,-0.321516,0.992197,-0.539598,0
BDX,Beldex,CryptoNight,PoW,980.222595,1400.222610,0.314666,-2.399347,0.466314,3
ZEN,Horizen,Equihash,PoW,7.296538,21.000000,-0.165754,-1.807345,0.322140,3


In [38]:
# Print the total number of tradable cryptocurrencies
count = clustered_df['CoinName'].count()
print(f"There are {count} tradable cryptocurrencies")


There are 532 tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [36]:
# Scale data to create the scatter plot
clustered_df['TotalCoinsMined'] = clustered_df['TotalCoinsMined'].astype(float) / 1000000
clustered_df['TotalCoinSupply'] = clustered_df['TotalCoinSupply'].astype(float) / 1000000



In [37]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
alt.Chart(clustered_df).mark_circle(size=60).encode(
    x="TotalCoinsMined", 
    y="TotalCoinSupply",
)

alt.Chart(...)